In [3]:
import os
import zipfile
import numpy as np
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import ConvLSTM2D, MaxPooling3D, TimeDistributed, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping



In [27]:

# Paths
dataset_path = "/home/fahim/Music/CarsDataset.zip"
extracted_path = "/home/fahim/Music/CarsDataset"

# Extract dataset
if not os.path.exists(extracted_path):
    with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_path)
else:
    print(f"Dataset already extracted at {extracted_path}")


Dataset already extracted at /home/fahim/Music/CarsDataset


In [4]:
# Function to load and preprocess images
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    try:
        img = Image.open(image_path)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img = img.resize(target_size)
        img_array = np.array(img) / 255.0
        return img_array
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None



In [5]:
# Function to create data combinations
def create_data_combinations(image_folder):
    images, labels = [], []
    class_mapping = {
        'Audi': 0,
        'Rolls Royce': 1,
        'Toyota_celica': 2,
        'Toyota_hilux': 3,
        'Toyota_supra': 4,
        'BMW': 5,
        'Swift': 6,
        'Toyota_corolla': 7,
        'Toyota Innova': 8,
        'Toyota_yaris': 9,
        'Hyundai Creta': 10,
        'Tata Safari': 11,
        'Toyota_corona': 12,
        'Toyota_prius': 13,
        'Lamborghini': 14,
        'Toyota_alphard': 15,
        'Toyota_crown': 16,
        'Toyota_rav4': 17,
        'Mahindra Scorpio': 18,
        'Toyota_avanza': 19,
        'Toyota_fortuner': 20,
        'Toyota_rush': 21,
        'Mercedes': 22,
        'Toyota_camry': 23,
        'Toyota_hiace': 24
    }

    for class_folder in os.listdir(image_folder):
        class_path = os.path.join(image_folder, class_folder)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            img_data = load_and_preprocess_image(image_path)
            if img_data is not None:
                images.append(img_data)
                label = class_mapping[class_folder]
                labels.append(label)
        print(f"Loaded images from class: {class_folder}")
    return images, labels


In [6]:
# image_folder = os.path.join(extracted_path, 'train')

image_folder = '/home/fahim/Music/CarsDataset/train'
images, labels = create_data_combinations(image_folder)






Loaded images from class: Toyota_supra
Loaded images from class: Toyota_hilux
Loaded images from class: Audi
Loaded images from class: Toyota_rav4
Loaded images from class: Toyota_corona
Loaded images from class: Toyota_camry
Loaded images from class: Toyota_alphard
Loaded images from class: Rolls Royce
Loaded images from class: Toyota_corolla
Loaded images from class: Tata Safari
Loaded images from class: Swift
Loaded images from class: Toyota_hiace
Loaded images from class: BMW
Loaded images from class: Toyota_prius
Loaded images from class: Toyota_crown
Loaded images from class: Toyota Innova
Loaded images from class: Toyota_rush
Loaded images from class: Toyota_fortuner
Loaded images from class: Mercedes
Loaded images from class: Toyota_yaris
Loaded images from class: Hyundai Creta
Loaded images from class: Lamborghini
Loaded images from class: Toyota_avanza
Loaded images from class: Toyota_celica
Loaded images from class: Mahindra Scorpio


In [7]:
X_image = np.array(images)
Y_labels = np.array(labels)
X_image = np.expand_dims(X_image, axis=1)

print("Shape of X_image:", X_image.shape)
print("Shape of Y_labels:", Y_labels.shape)

Shape of X_image: (1080, 1, 224, 224, 3)
Shape of Y_labels: (1080,)


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X_image, Y_labels, test_size=0.2, random_state=42)


In [9]:
def create_convlstm_model(input_shape, class_mapping_length):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(ConvLSTM2D(filters=4, kernel_size=(3, 3), activation='tanh', return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2)))
    model.add(TimeDistributed(Dropout(0.2)))
    model.add(ConvLSTM2D(filters=8, kernel_size=(3, 3), activation='tanh', return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2)))
    model.add(TimeDistributed(Dropout(0.2)))
    model.add(ConvLSTM2D(filters=14, kernel_size=(3, 3), activation='tanh', return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2)))
    model.add(TimeDistributed(Dropout(0.2)))
    model.add(ConvLSTM2D(filters=16, kernel_size=(3, 3), activation='tanh', return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2)))
    model.add(Flatten())
    model.add(Dense(class_mapping_length, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

In [ ]:
# input_shape = X_train.shape[1:]
# model = create_convlstm_model(input_shape)
# model.summary()

# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=1, batch_size=32, callbacks=[early_stopping])

# # Evaluate model
# loss, accuracy = model.evaluate(X_test, Y_test)
# print(f"Test Accuracy: {accuracy * 100:.2f}%")

# model_path = "/home/fahim/Music/car_model_classifier_convlstm2.h5"
# model.save(model_path)
# print(f"Model saved at: {model_path}")

# # Function to preprocess locally uploaded image
# def preprocess_local_image(image_path):
#     try:
#         img = cv2.imread(image_path)
#         img = cv2.resize(img, (224, 224))  # Resize image
#         img = img.astype('float32') / 255.0
#         return img
#     except Exception as e:
#         print("Error:", str(e))
#         return None

# # Function to predict car model from a local image file
# def predict_car_model_local(image_path, model, class_mapping):
#     preprocessed_image = preprocess_local_image(image_path)
#     if preprocessed_image is None:
#         return None
    
#     preprocessed_image = np.expand_dims(preprocessed_image, axis=0)
#     preprocessed_image = np.expand_dims(preprocessed_image, axis=1)
#     prediction = model.predict(preprocessed_image)
#     predicted_label_index = np.argmax(prediction)
#     return class_mapping.get(predicted_label_index, "Unknown")

# class_mapping = {
#     0: 'Audi',
#     1: 'Rolls Royce',
#     2: 'Toyota_celica',
#     3: 'Toyota_hilux',
#     4: 'Toyota_supra',
#     5: 'BMW',
#     6: 'Swift',
#     7: 'Toyota_corolla',
#     8: 'Toyota Innova',
#     9: 'Toyota_yaris',
#     10: 'Hyundai Creta',
#     11: 'Tata Safari',
#     12: 'Toyota_corona',
#     13: 'Toyota_prius',
#     14: 'Lamborghini',
#     15: 'Toyota_alphard',
#     16: 'Toyota_crown',
#     17: 'Toyota_rav4',
#     18: 'Mahindra Scorpio',
#     19: 'Toyota_avanza',
#     20: 'Toyota_fortuner',
#     21: 'Toyota_rush',
#     22: 'Mercedes',
#     23: 'Toyota_camry',
#     24: 'Toyota_hiace'
# }

# uploaded_image_path = '/home/fahim/Downloads/rr3.jpeg'  # Replace with your image path
# model = load_model(model_path)
# predicted_car_model = predict_car_model_local(uploaded_image_path, model, class_mapping)

# if predicted_car_model is not None:
#     print("Predicted Car Model:", predicted_car_model)
# else:
#     print("Prediction failed. Please check your image path.")

In [10]:
class_mapping = {
    'Audi': 0,
    'Rolls Royce': 1,
    'Toyota_celica': 2,
    'Toyota_hilux': 3,
    'Toyota_supra': 4,
    'BMW': 5,
    'Swift': 6,
    'Toyota_corolla': 7,
    'Toyota Innova': 8,
    'Toyota_yaris': 9,
    'Hyundai Creta': 10,
    'Tata Safari': 11,
    'Toyota_corona': 12,
    'Toyota_prius': 13,
    'Lamborghini': 14,
    'Toyota_alphard': 15,
    'Toyota_crown': 16,
    'Toyota_rav4': 17,
    'Mahindra Scorpio': 18,
    'Toyota_avanza': 19,
    'Toyota_fortuner': 20,
    'Toyota_rush': 21,
    'Mercedes': 22,
    'Toyota_camry': 23,
    'Toyota_hiace': 24
}


In [20]:
model_path = "/home/fahim/Music/car_model_classifier_convlstm2.h5"

In [21]:
from tensorflow.keras.layers import Input
# Load the pre-trained model or train a new one
if os.path.exists(model_path):
    print("Loading pre-trained model from:", model_path)
    model = load_model(model_path)
else:
    print("No pre-trained model found. Training a new model.")
    input_shape = X_train.shape[1:]
    class_mapping_length = len(class_mapping)
    model = create_convlstm_model(input_shape, class_mapping_length)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, batch_size=32, callbacks=[early_stopping])
    model.save(model_path)
    print(f"Model saved at: {model_path}")

No pre-trained model found. Training a new model.
Epoch 1/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 400ms/step - accuracy: 0.0474 - loss: 3.2178 - val_accuracy: 0.0324 - val_loss: 3.2033
Epoch 2/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 381ms/step - accuracy: 0.0480 - loss: 3.1930 - val_accuracy: 0.0278 - val_loss: 3.1918
Epoch 3/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 381ms/step - accuracy: 0.0402 - loss: 3.1936 - val_accuracy: 0.0278 - val_loss: 3.1859
Epoch 4/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 380ms/step - accuracy: 0.0372 - loss: 3.1787 - val_accuracy: 0.0278 - val_loss: 3.1849
Epoch 5/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 382ms/step - accuracy: 0.0446 - loss: 3.1772 - val_accuracy: 0.0278 - val_loss: 3.1838
Epoch 6/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 382ms/step - accuracy: 0.0404 - loss: 3.1794 - val_accuracy: 0.0278 - val_loss: 3.1846
Epoch 7/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 383ms/step - accuracy: 0.0444 - loss: 3.1779 - val_accuracy: 0.0278 - val_loss: 3.1788
Epoch 8/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 1

Model saved at: /home/fahim/Music/car_model_classifier_convlstm2.h5


In [22]:

# Evaluate model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")



7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.0974 - loss: 3.0343
Test Accuracy: 9.72%


In [23]:
# Function to preprocess locally uploaded image
def preprocess_local_image(image_path):
    try:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (224, 224))  # Resize image
        img = img.astype('float32') / 255.0
        return img
    except Exception as e:
        print("Error:", str(e))
        return None

# Function to predict car model from a local image file
def predict_car_model_local(image_path, model, class_mapping):
    preprocessed_image = preprocess_local_image(image_path)
    if preprocessed_image is None:
        return None
    
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)
    preprocessed_image = np.expand_dims(preprocessed_image, axis=1)
    prediction = model.predict(preprocessed_image)
    predicted_label_index = np.argmax(prediction)
    return class_mapping.get(predicted_label_index, "Unknown")

class_mapping = {
    0: 'Audi',
    1: 'Rolls Royce',
    2: 'Toyota_celica',
    3: 'Toyota_hilux',
    4: 'Toyota_supra',
    5: 'BMW',
    6: 'Swift',
    7: 'Toyota_corolla',
    8: 'Toyota Innova',
    9: 'Toyota_yaris',
    10: 'Hyundai Creta',
    11: 'Tata Safari',
    12: 'Toyota_corona',
    13: 'Toyota_prius',
    14: 'Lamborghini',
    15: 'Toyota_alphard',
    16: 'Toyota_crown',
    17: 'Toyota_rav4',
    18: 'Mahindra Scorpio',
    19: 'Toyota_avanza',
    20: 'Toyota_fortuner',
    21: 'Toyota_rush',
    22: 'Mercedes',
    23: 'Toyota_camry',
    24: 'Toyota_hiace'
}



In [24]:
uploaded_image_path = '/home/fahim/Downloads/rr2.jpeg'  # Replace with your image path
predicted_car_model = predict_car_model_local(uploaded_image_path, model, class_mapping)

if predicted_car_model is not None:
    print("Predicted Car Model:", predicted_car_model)
else:
    print("Prediction failed. Please check your image path.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step
Predicted Car Model: Toyota_fortuner


In [25]:
import cv2

img = cv2.imread('/home/fahim/Downloads/rr.jpeg')
if img is not None:
    print("Image loaded successfully.")
else:
    print("Failed to load image.")


Image loaded successfully.


In [1]:
import cv2
print(cv2.__version__)


4.6.0


In [60]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.
